In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from typing import List
from collections import defaultdict

In [2]:
# fname = "../data/huang_emt_scc.gxml"
# fname = "../data/omnipath.gxml"
fname = "../data/tcell_scc.gxml"
g = nx.read_graphml(fname)

# Network Cycles becomes Nodes

## Filtering for only Positive-cycles
A cycle is positive if the XOR of their "represses" of all the edges results in a positive

In [3]:
cycles = sorted(nx.simple_cycles(g))
cycle = cycles[0]
positive = True
print(positive, end ="")
for i, node in enumerate(cycle):
    next_node = cycle[(i + 1) % len(cycle)]
    repress = g.edges[node, next_node]['repress']
    positive ^= repress
    print(f" -> ({node}, {next_node}: {repress}) -> {positive}", end="")
    if (i + 1) % 3 == 0:
        print(f'\n{positive}', end="")

True -> (0, 11: False) -> True -> (11, 22: True) -> False -> (22, 15: True) -> True
True -> (15, 7: False) -> True -> (7, 19: False) -> True -> (19, 13: True) -> False
False -> (13, 3: False) -> False -> (3, 0: False) -> False

In [4]:
def is_positive(graph: nx.Graph, cycle: List[str]):
    positive = True
    for i, node in enumerate(cycle):
        next_node = cycle[(i + 1) % len(cycle)]
        repress = g.edges[node, next_node]['repress']
        positive ^= repress
    return positive

Find all positive cycles

In [5]:
all_cycles = list(nx.simple_cycles(g))
cycles = [c for c in nx.simple_cycles(g) if is_positive(g, c)]
len(all_cycles) - len(cycles)

1424

## Edge to Cycles Map
For each edge, create a set that tracks all the cycles that the edge is in

In [6]:
edge_cycles_mp = defaultdict(set)
for cycle in all_cycles:
    cycle_set = frozenset(cycle)
    for i, node in enumerate(cycle):
        next_node = cycle[(i + 1) % len(cycle)]
        edge_cycles_mp[node, next_node].add(cycle_set)
nx.set_edge_attributes(g, edge_cycles_mp, "cycles")

## Node to Cycles Map
For each node, create a set that tracks all the cycles that the node is in

In [7]:
node_cycles_mp = defaultdict(set)
for cycle in all_cycles:
    cycle_set = frozenset(cycle)
    for _, node in enumerate(cycle):
        node_cycles_mp[node].add(cycle_set)
        
nx.set_node_attributes(g, node_cycles_mp, "cycles")

# for n, c in g.nodes(data='cycles'):
#     print(n, len(c))

# Cycle Intersection Graph

In [76]:
frozen_cycles = [frozenset(c) for c in all_cycles]
positive_cycles = [is_positive(g, c) for c in all_cycles]
cycle_graph = nx.Graph()
for fc, pc in zip(frozen_cycles, positive_cycles):
    cycle_graph.add_node(fc, positive=pc)

In [77]:
for i, c1 in enumerate(frozen_cycles):
    for c2 in frozen_cycles[i + 1:]:
        common = c1 & c2
        if common:
            cycle_graph.add_edge(c1, c2, nodes=common)

In [78]:
cycle_graph.nodes()[(frozenset({'8'}))]

{'positive': True}

In [69]:
len(cycle_graph.edges)

1446619

## Type 1 Detection
Three cycles with a common node

In [ ]:
for edge, cycle_set in edge_cycles_mp.items():
    print(edge, cycle_set)

# Type 2 Detection
Type 2 motifs are two cycles connected by a third cycle